<a href="https://colab.research.google.com/github/haru1489248/nlp-100-nock/blob/main/ch08/section_74.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 74. 開発セットにおける正解率を求める
- 必要なもの
  - dev.tsv
  - 73と同じkey_to_idxとembedding_matrixを用意
  - devをtoken化してDataset / DataLoaderを作成する

In [ ]:
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from typing import Any, Dict, List, Set, Tuple, Union
from torch.utils.data import Dataset, DataLoader
from google.colab import drive
from tqdm.notebook import tqdm
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 73でsaveしたファイルを読み込む
bundle = torch.load(
    "/content/drive/MyDrive/section_73_bundle.pth",
    map_location=device
)

In [ ]:
key_to_idx = bundle["key_to_idx"]
# Dataset.__getitem__ は CPU 上で実行されるため、
# input_ids は常に CPU Tensor になる。
# embedding_matrix が GPU Tensor だと
# embedding_matrix[input_ids] で device mismatch が起きるため、
# embedding_matrix は明示的に CPU に置いておく。
embedding_matrix = bundle["embedding_matrix"].to("cpu")
embedding_dim = bundle["meta"]["embedding_dim"]
threshold = bundle["meta"]["threshold"]

In [ ]:
class SSTDataset(Dataset):
    """
    Dataset Class for the SST-2.
    """

    def __init__(self, data: List[Dict[str, torch.Tensor]], embedding_matrix: torch.Tensor) -> None:
        super().__init__() # ほとんど何もしていないらしいが継承クラスを初期化する慣習
        self.data = data
        self.embedding_matrix = embedding_matrix

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, index: int) -> Tuple[torch.Tensor, torch.Tensor]:
        object = self.data[index]
        input_ids = object["input_ids"]
        embeddings = self.embedding_matrix[input_ids]

        # 平均化ベクトルの取得
        # torch.meanはtensorに含まれるすべての要素の平均を返す
        # dimを0と指定すると列方向に平均、1と指定すると行方向に平均をとる
        mean_embedding = torch.mean(embeddings, dim=0)
        return mean_embedding, object["label"]

In [ ]:
class SemanticClassifier(nn.Module):
    """
    Bag of words.
    """

    def __init__(self, in_dimension: int, n_classes: int) -> None:
        super().__init__()
        self.in_dimension = in_dimension
        self.n_classes = n_classes
        self.linear1 = nn.Linear(in_features=in_dimension, out_features=1, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # squeeze(1) は「index が 1 の軸のサイズが 1 のときだけ、その軸を削除して次元を 1 つ減らす」
        return self.sigmoid(self.linear1(x)).squeeze(1)

In [ ]:
def fix_seeds(seed: int) -> None:
    """Fix seeds, Pytorch, random, numpy.

    Parameters
    ----------
    seed : int
        Number of a seed.
    """
    # random.random()・random.shuffle()などに影響する
    # 今回は使用されていなそうだが慣習で設定するようにしている
    random.seed(seed)

    # おそらく間違いで書かれていたコード
    # これはseed付き乱数生成器を作っているだけで使っていない（インスタンス化している）
    # npのseedは使われていなかったので大丈夫だった
    np.random.RandomState(seed)
    #正解はグローバルなnumpyの乱数を固定するために定義したこれ
    np.random.seed(seed)

    # PyTorch（CPU）の乱数を固定する
    torch.manual_seed(seed)
    # GPUの場合
    torch.cuda.manual_seed(seed)

    # GPU内で同じ入力の場合に同じ出力を保証する設定
    # cuDNNはNVIDIAが提供するGPU用の高速数値計算ライブラリ
    # PyTorchでは内部でこれを使用してLinear、Conv、RNNなどを高速化している
    # 計算速度を多少遅くするが結果が必ず同じアルゴリズムだけを使用するようにできる
    torch.backends.cudnn.deterministic = True

In [ ]:
def tokenize(row: pd.Series, key_to_idx: Dict[str, int]) -> Tuple[Dict[str, Any], int]:
    """Convert inputted text and label to dict object.

    Parameters
    ----------
    row : pd.Series
        Row of the dataset.
    key_to_idx : Dict[str, int]
        Dictionary of word to index.
    Returns
    -------
    Tuple[Dict[str, Any], int]
        Tokenized data dictionary and token count
    """
    sentence = row["sentence"]
    label = row["label"]
    input_ids = []

    for word in sentence.lower().split():
        if word in key_to_idx:
            input_ids.append(key_to_idx[word])

    token_dict = {
        "text": sentence,
        "label": torch.tensor(label, dtype=torch.long),
        "input_ids": torch.tensor(input_ids, dtype=torch.long),
    }

    return token_dict, len(input_ids)


In [ ]:
def convert_to_token(df: pd.DataFrame, key_to_idx: Dict[str, int]) -> List[Dict[str, torch.Tensor]]:
    """Apply tokenize function to each row of the dataframe.

    Parameters
    ----------
    df : pd.DataFrame
        Dataset dataframe.
    key_to_idx : Dict[str, int]
        Dictionary of word to index.

    Returns
    -------
    List[Dict[str, torch.Tensor]]
        List of tokenized data dictionaries
    """
    # sentenceのindex化
    # argsとはapplyで呼ばれる関数にrow以外で追加で渡したい引数を指定するもの
    # tupleで指定している理由
    # argsはtupleで渡さないといけないので(key_to_idx,)としている:(key_to_idx)だとint
    tokenized_data = df.apply(tokenize, args=(key_to_idx,), axis=1)

    # token数が0の行を除く
    #.token_count = input_idsで単語ベクトルが存在しないものをフィルターしている
    result = [token_dict for token_dict, token_count in tokenized_data if token_count > 0]

    return result


In [ ]:
def dev(
    model: nn.Module,
    devloader: DataLoader,
    criterion: nn.BCELoss,
    device: torch.device,
    threshold: float,
):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    num_batches = 0

    with torch.no_grad():
        for mean_embedding, label in tqdm(devloader, desc="Evaluating"):
            mean_embedding = mean_embedding.to(device)
            label = label.to(device).to(torch.float32)

            pred = model(mean_embedding)
            loss = criterion(pred, label)

            pred_binary = (pred >= threshold).float()
            correct += (pred_binary == label).sum().item()
            total += label.size(0)

            total_loss += loss.item()
            num_batches += 1

    avg_loss = total_loss / num_batches if num_batches > 0 else 0.0
    acc = correct / total if total > 0 else 0.0
    # set_postfixは進捗バーの横に出力されるが
    # writeは新しく行で出力される
    tqdm.write(f"loss so far: {avg_loss:.4f}")
    tqdm.write(f"acc so far: {acc:.4f}")
    return avg_loss, acc


In [ ]:
def main(args) -> None:
    fix_seeds(args.seed)
    dev_df = pd.read_csv("/content/drive/MyDrive/SST-2/dev.tsv", sep="\t")
    dev_data = convert_to_token(dev_df, key_to_idx)
    dev_dataset = SSTDataset(dev_data, embedding_matrix)
    dev_loader = DataLoader(dev_dataset, batch_size=args.batch_size, shuffle=False)
    device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
    model = SemanticClassifier(in_dimension=embedding_matrix.size(1), n_classes=2).to(device)

    # 73で学習したモデルをロードする
    model.load_state_dict(bundle["model_state_dict"])
    criterion = nn.BCELoss()
    avg_loss, avg_accuracy = dev(
            model=model,
            devloader=dev_loader,
            criterion=criterion,
            device=device,
            threshold=threshold,
        )
    print(f"final loss: {avg_loss:.4f}")
    print(f"final accuracy: {avg_accuracy:.4f}")


In [ ]:
if __name__ == "__main__":
    # pythonスクリプトをコマンドラインから実行するときに渡された引数（args）を定義通りに解釈・解析（parse）するための# 標準ライブラリ
    import argparse

    # ArgumentParserのインスタンスを作成する（argumentを登録するために必要）
    parser = argparse.ArgumentParser()

    # add_argument(短い名前（省略可）, 長い名前, type・defaultなどの解釈ルール設定)
    parser.add_argument("-s", "--seed", type=int, default=29)
    parser.add_argument("-e", "--epochs", default=100, type=int)
    parser.add_argument("-b", "--batch_size", default=32, type=int)
    parser.add_argument("-p", "--postfix", type=str) # 今の所使用していない

    # store_trueはそのオプションが指定されたらTrue、指定されなかったらFalseにする
    parser.add_argument("--dryrun", action="store_true")

    # 内部でimport sysをしているのでimport不要
    # sys.argvを読んで登録したルールに従って解析をする
    # sys.argvの出力例
    # ['train.py', '-e', '50', '--dryrun']
    # colabでは特有のargsを渡してくるので定義したargsのみ受け取るようにする
    args, unknown = parser.parse_known_args()
    main(args)